# worldometers table scraping

## extracting text from html table

In [1]:
import requests
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
from pprint import pprint

fileDay = datetime.date.today().strftime('%m%d')

df = None

colList = []  
country = []
totCase = []
newCase = []
totDeath = []
newDeath = []
totRecov = []
actCase = []
serious = []
totCasePer1M = []
totDeathPer1M = [] # this column was added in 2020. 3. 25.
# firstCase = [] # this column was added in 2020. 3. 28. # this column was removed in 2020. 4. 3.
totTest = [] # this column was added in 2020. 4. 4.
totTestPer1M = [] # this column was added in 2020. 4. 4.
continent = [] # this column was added in 2020. 4 . 9.

allList = [colList, country, totCase, newCase, totDeath, newDeath, totRecov, actCase, serious, totCasePer1M, totDeathPer1M, totTest, totTestPer1M, continent]

def colClear():
    for i in allList:
        i.clear()

def scrapingWorldometers(yda=False):
    url = "https://www.worldometers.info/coronavirus/"
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    
    if yda:
        table = soup.select_one('table#main_table_countries_yesterday')
    else:
        table = soup.select_one('table#main_table_countries_today')
    #pprint(table)

    columns = table.select('tr')
    isCol = True
    colClear()

    for col in columns:
        if isCol:
            cols = col.select('th')
            for c in cols:
                colList.append(c.text)
            isCol = False
        else:
            rows = col.select('td')
            country.append(rows[0].text.strip())
            totCase.append(rows[1].text.strip())
            newCase.append(rows[2].text.strip())
            totDeath.append(rows[3].text.strip())
            newDeath.append(rows[4].text.strip())
            totRecov.append(rows[5].text.strip())
            actCase.append(rows[6].text.strip())
            serious.append(rows[7].text.strip())
            totCasePer1M.append(rows[8].text.strip())
            totDeathPer1M.append(rows[9].text.strip())
            #firstCase.append(rows[10].text.strip())
            totTest.append(rows[10].text.strip())
            totTestPer1M.append(rows[11].text.strip())
            continent.append(rows[12].text.strip())
    #print(colList)

## making dataframe, json, csv

In [2]:
def dfProcessing():
    # 'Tot\xa0Cases/1M pop' => 'Tot Cases/1M pop'
    # 'Tot\xa0Deaths/1M pop' => 'Tot Deaths/1M pop' => Deaths/1M pop(3. 30)
    # 'Tests/\n1M pop\n' => 'Tests/1M pop'
    colList[8] = 'Tot Cases/1M pop'
    colList[9] = 'Deaths/1M pop'
    colList[11] = 'Tests/1M pop'
    
    # Réunion, Curaçao => Reunion, Curacao 
    idx = country.index("Réunion")
    country[idx] = "Reunion"
    
    idx = country.index("Curaçao")
    country[idx] = "Curacao"
    
    # '\nJan 20 ' => 'Jan 20'
    #for day in firstCase:
        #idx = firstCase.index(day)
        #nday = day.strip()
        #firstCase[idx] = nday
    
    df = pd.DataFrame(columns=colList)
    df.iloc[:,0] = country
    df.iloc[:,1] = totCase
    df.iloc[:,2] = newCase
    df.iloc[:,3] = totDeath
    df.iloc[:,4] = newDeath
    df.iloc[:,5] = totRecov
    df.iloc[:,6] = actCase
    df.iloc[:,7] = serious
    df.iloc[:,8] = totCasePer1M
    df.iloc[:,9] = totDeathPer1M
    #df.iloc[:,10] = firstCase
    df.iloc[:,10] = totTest
    df.iloc[:,11] = totTestPer1M
    df.iloc[:,12] = continent
    
    df = df.set_index('Country,Other')
    
    df = df.replace(',','', regex=True)
    df = df.replace('', 0)
    df = df.replace(' ', 0)
    
    # Europe recov / cases * UK case 2020. 4. 13. ~ 17.(continue)
    df.loc["UK", "TotalRecovered"] = round(int(df.loc["Europe", "TotalRecovered"]) / int(df.loc["Europe", "TotalCases"]) * int(df.loc["UK", "TotalCases"]))

    astypeDict = dict.fromkeys(df.columns, np.int)
    astypeDict['Tot Cases/1M pop'] = np.float
    astypeDict['Deaths/1M pop'] = np.float
    astypeDict['Tests/1M pop'] = np.float
    #astypeDict['Reported1st case'] = np.string_ # 1stcase => Reported1st case (3. 30)
    astypeDict['Continent'] = np.string_

    df.astype(astypeDict).dtypes
    return df

def toCSV(df, yda=False):
    if yda:
        csvFile = './covidDataset/' + fileDay + 'ydaCoronaWorld.csv'
        df.to_csv(csvFile)
        print(fileDay, "yesterday csv complete")
    else:
        csvFile = './covidDataset/' + fileDay + 'tdaCoronaWorld.csv'
        df.to_csv(csvFile)
        print(fileDay, "today csv complete")
    
def toJSON(df, yda=False, orient='table'):
    if yda:
        jsonFile = './covidDataset/' + fileDay + 'yda' + orient + 'CoronaWorld.json'
        df.to_json(jsonFile, orient)
        print(fileDay, "yesterday json complete")
    else:
        jsonFile = './covidDataset/' + fileDay + 'tda' + orient + 'CoronaWorld.json'
        df.to_json(jsonFile, orient)
        print(fileDay, "today json complete")

## today files

In [3]:
scrapingWorldometers()
df = dfProcessing()
toCSV(df)
toJSON(df)
toJSON(df, orient='records')
#toJSON(df, orient='index') # 2020. 4. 10. Total row duplicated owing to regions
#toJSON(df, orient='columns')

0417 today csv complete
0417 today json complete
0417 today json complete


## yesterday files

In [4]:
scrapingWorldometers(yda=True)
df = dfProcessing()
toCSV(df, yda=True)
toJSON(df, yda=True)
toJSON(df, yda=True, orient='records')
#toJSON(df, yda=True, orient='index')
#toJSON(df, yda=True, orient='columns')

0417 yesterday csv complete
0417 yesterday json complete
0417 yesterday json complete


In [5]:
#scrapingWorldometers()
print(colList)

['Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'ActiveCases', 'Serious,Critical', 'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop', 'Continent']
